In [4]:
#installing beautifulsoup
!pip install --upgrade beautifulsoup4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.0/143.0 kB 2.0 MB/s eta 0:00:00
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.11.2
    Uninstalling beautifulsoup4-4.11.2:
      Successfully uninstalled beautifulsoup4-4.11.2


In [5]:
# 1. import libraries
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re

In [ ]:
# 2. find url and store it in a variable
url = "https://en.wikipedia.org/wiki/Berlin"

In [ ]:
# 3. download html with a get request
response = requests.get(url)
response.status_code # 200 status code means OK!

200

In [ ]:
# 4.1. parse html (create the 'soup')
soup = BeautifulSoup(response.content, "html.parser")

In [ ]:
# let's first try to get the name of the city
soup.select("#firstHeading")

[<h1 class="firstHeading mw-first-heading" id="firstHeading"><span class="mw-page-title-main">Berlin</span></h1>]

In [ ]:
#selecting first element of "FirstHeading"
soup.select("#firstHeading")[0]

<h1 class="firstHeading mw-first-heading" id="firstHeading"><span class="mw-page-title-main">Berlin</span></h1>

In [ ]:
#collecting as a text
soup.select("#firstHeading")[0].get_text()

'Berlin'

In [ ]:
#Use infobox-data class for searching our needed informations.
soup.select(".infobox-data")

In [ ]:
#selecting first element of class
soup.select(".infobox-data")[0]

<td class="infobox-data"><a href="/wiki/Germany" title="Germany">Germany</a></td>

In [ ]:
# Let's use this class, infobox-data, to target the information country
soup.select(".infobox-data")[0].get_text()

'Germany'

In [ ]:
# Population
soup.select(".infobox-data")[10].get_text()

'3,850,809'

In [ ]:
# Latitude
soup.select(".latitude")[0].get_text()

'52°31′12″N'

In [ ]:
# Longitude
soup.select(".longitude")[0].get_text()

'13°24′18″E'

In [6]:
#creating function for collect multiple cities data.
def recreate_wiki(cities):
  # empty list that will be filled with one dictionary of information per city
  list_for_df = []

  # begin a for loop to create a dictionary of information for each city
  for city in cities:
    # we can use the universal nature of wikipedias urls to our advantage here
    # all of the urls are the same besides the city name
    url = f'https://en.wikipedia.org/wiki/{city}'

    # here we make our soup for the city
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'html.parser')

    # here we initialise our empty dictionary for the city
    response_dict = {}

    # here we fill the dictionary with information using the ids, classes, and selectors that we found in the html
    response_dict['city'] = soup.select(".firstHeading")[0].get_text()
    response_dict['country'] = soup.select(".infobox-data")[0].get_text()
    response_dict['latitude'] = soup.select(".latitude")[0].get_text()
    response_dict['longitude'] = soup.select(".longitude")[0].get_text()

    # the if clause means that our code can continue and won't stop at this hurdle
    if soup.select_one('th.infobox-header:-soup-contains("Population")'):
        response_dict['population'] = soup.select_one('th.infobox-header:-soup-contains("Population")').parent.find_next_sibling().find(text=re.compile(r'\d+'))

    # add our dictionary for the city to list_for_df
    list_for_df.append(response_dict)

  # make the DataFrame
  cities_df = pd.DataFrame(list_for_df)

  # fixing latitude
  cities_df['latitude'] = cities_df['latitude'].str.split('″').str[0].str.replace('°', '.', regex=False).str.replace('′', '', regex=False)
  # fixing longitude
  cities_df['longitude'] = cities_df['longitude'].str.split('″').str[0].str.replace('°', '.', regex=False).str.replace('′', '', regex=False)

  # return the DataFrame
  return cities_df

In [8]:
#Applying for two cities such as Berlin and Istanbul
list_of_cities = ['Berlin', 'Istanbul']
cities_df = recreate_wiki(list_of_cities)
cities_df

<ipython-input-6-a9a58105e67e>:27: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  response_dict['population'] = soup.select_one('th.infobox-header:-soup-contains("Population")').parent.find_next_sibling().find(text=re.compile(r'\d+'))


,city,country,latitude,longitude,population
0,Berlin,Germany,52.3112,13.2418,"3,850,809"
1,Istanbul,Turkey,41.0049,28.5718,"15,907,951"


In [9]:
#creating index for city id
cities_df['city_id'] = cities_df.index
cities_df

,city,country,latitude,longitude,population,city_id
0,Berlin,Germany,52.3112,13.2418,"3,850,809",0
1,Istanbul,Turkey,41.0049,28.5718,"15,907,951",1


In [ ]:
# create csv
cities_df.to_csv('cities.csv')

In [ ]:
# import library
from google.colab import files
# download csv
files.download('cities.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>